In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
import warnings

In [2]:
# Load your dataset
df = pd.read_csv('../raw_data/merge_df_resize.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

df.head() # 382600 rows × 73 columns

,id,item_id,dept_id,state_id,sales,wday,month,year,event_name_2,snap_CA,...,event_name_1_StPatricksDay,event_name_1_SuperBowl,event_name_1_Thanksgiving,event_name_1_ValentinesDay,event_name_1_VeteransDay,event_name_1_missing,wday_sin,wday_cos,month_sin,month_cos
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,FOODS_2_197_CA_1_validation,FOODS_2_197,FOODS_2,CA,38,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025
2011-01-29,FOODS_3_080_CA_1_validation,FOODS_3_080,FOODS_3,CA,33,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025
2011-01-29,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,CA,107,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025
2011-01-29,FOODS_3_120_CA_1_validation,FOODS_3_120,FOODS_3,CA,0,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025
2011-01-29,FOODS_3_252_CA_1_validation,FOODS_3_252,FOODS_3,CA,19,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025


# 1.Auto_ARIMA with mae

In [3]:
from pmdarima import auto_arima

def train_arima_model(product_data):
    # Split data into training and test sets
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    y_train = data_train["sales"]
    y_test = data_test["sales"]

    # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
    model = auto_arima(y_train, start_p=0, start_q=0, max_p=5, max_q=5, d=1,
                       seasonal=True, trace=False, error_action='ignore', 
                       suppress_warnings=True, stepwise=True)

    # Predict on the test data
    predictions = model.predict(n_periods=len(y_test))

    # Calculate and return the error metric
    mae = mean_absolute_error(y_test, predictions)
    return mae

# Dictionary to store MAE results for each unique time-series identified by id
product_results = {}

# Iterate over each unique product series identified by id 
for id in df['id'].unique()[:10]:
    print(f"Analyzing product: {id}")
    product_data = df[df['id'] == id]

    # Call the function to train and evaluate the ARIMA model
    mae = train_arima_model(product_data)
    
    # Store the result in the dictionary
    product_results[id] = mae
    print(f'Mean Absolute Error for {id}: {mae}')

# Create a DataFrame to store the results
results_df_arima = pd.DataFrame(product_results.items(), columns=['id', 'ARIMA_MAE'])

# Set the 'id' column as the index
results_df_arima.set_index('id', inplace=True)


Analyzing product: FOODS_2_197_CA_1_validation
Mean Absolute Error for FOODS_2_197_CA_1_validation: 8.726633436944372
Analyzing product: FOODS_3_080_CA_1_validation
Mean Absolute Error for FOODS_3_080_CA_1_validation: 6.034984090618189
Analyzing product: FOODS_3_090_CA_1_validation
Mean Absolute Error for FOODS_3_090_CA_1_validation: 19.479598099076547
Analyzing product: FOODS_3_120_CA_1_validation


KeyboardInterrupt: 

In [ ]:
results_df_arima 

# 2.Holt-Winters Exponential Smoothing model with mae 

In [ ]:
import optuna

def train_exponential_smoothing_model(y_train, data_test):
    # Define the objective function for hyperparameter optimization
    def objective(trial):
        trend = trial.suggest_categorical('trend', ['add'])
        seasonal = trial.suggest_categorical('seasonal', [None, 'add'])
        seasonal_periods = trial.suggest_categorical('seasonal_periods', [None, 4, 7, 12])

        # Fit Exponential Smoothing model on the training data
        model = ExponentialSmoothing(y_train, trend=trend, seasonal=seasonal, 
                                      seasonal_periods=seasonal_periods, freq='D')
        fitted_model = model.fit(optimized=True)

        # Predict on the test data
        predictions = fitted_model.forecast(steps=len(data_test))

        # Calculate and return the error metric
        mae = mean_absolute_error(data_test, predictions)
        return mae

    # Create a study object
    study = optuna.create_study(direction='minimize')

    # Run the optimization process for the current product
    study.optimize(objective, n_trials=10)

    # Get the best hyperparameters and the corresponding best MAE
    best_params = study.best_params
    best_mae = study.best_value

    return best_params, best_mae

# Dictionary to store results for each unique time-series identified by id
results_dict = {}

# Iterate over each unique product series identified by id 
for id in df['id'].unique()[:10]:
    print(f"Optimizing hyperparameters for product: {id}")
    product_data = df[df['id'] == id]
    
    # Split data into training and test sets
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    y_train = data_train["sales"]
    
    # Call the function to train and evaluate the Exponential Smoothing model
    best_params, best_mae = train_exponential_smoothing_model(y_train, data_test["sales"])
    
    # Store the result in the dictionary
    results_dict[id] = {'ExpSmoothing_params': best_params, 'ExpSmoothing_MAE': best_mae}



In [ ]:
# Convert the results dictionary to a DataFrame
results_df_exp = pd.DataFrame(results_dict).T.reset_index()
results_df_exp.columns = ['id', 'ExpSmoothing_params', 'ExpSmoothing_MAE']

# Set the 'Product ID' column as the index
results_df_exp.set_index('id', inplace=True)
results_df_exp

In [ ]:
# import lightgbm as lgb

# def train_lightgbm_model(product_data):
#     # Split data into training and test sets
#     data_train = product_data.iloc[:-28]
#     data_test = product_data.iloc[-28:]
#     y_train = data_train["sales"]
#     y_test = data_test["sales"]

#     # Convert data into LightGBM dataset format
#     lgb_train = lgb.Dataset(np.array(y_train).reshape(-1, 1), label=y_train)
#     lgb_test = lgb.Dataset(np.array(y_test).reshape(-1, 1), label=y_test)

#     # Define parameters for the LightGBM model
#     params = {
#         'objective': 'regression',  # Regression task
#         'metric': 'mae',             # Mean Absolute Error
#         'verbosity': -1              # No output during training
#     }

#     # Train the LightGBM model with early stopping
#     gbm = lgb.train(params,
#                     lgb_train,
#                     num_boost_round=1000,      # Large number of boosting rounds
#                     valid_sets=[lgb_train, lgb_test],  # Validation data
#                     callbacks=[lgb.early_stopping(stopping_rounds=10)])   # Early stopping criterion

#     # Predict on the test set
#     y_pred = gbm.predict(np.array(y_test).reshape(-1, 1), num_iteration=gbm.best_iteration)
#     print(y_pred)

#     # Calculate and return the mean absolute error
#     mae = mean_absolute_error(y_test, y_pred)
#     return mae

# # Dictionary to store MAE results for each unique time-series identified by id
# product_results = {}

# # Iterate over each unique product series identified by id 
# for id in df['id'].unique()[:10]:
#     print(f"Analyzing product: {id}")
#     product_data = df[df['id'] == id]
#     mae = train_lightgbm_model(product_data)
#     product_results[id] = mae
#     print(f'Mean Absolute Error for {id}: {mae}')

# # Create a DataFrame to store the results
# results_df_lgbm = pd.DataFrame(product_results.items(), columns=['id', 'LightGBM_MAE'])

# # Set the 'id' column as the index
# results_df_lgbm.set_index('id', inplace=True)


In [ ]:
# results_df_lgbm

In [ ]:
# Merge the two DataFrames based on the product ID
comparison_df = pd.merge(results_df_arima, results_df_exp, left_index=True, right_index=True)
#comparison_df = pd.merge(comparison_df, results_df_lgbm, left_index=True, right_index=True)
comparison_df['Best MAE'] = comparison_df[['ARIMA_MAE', 'ExpSmoothing_MAE']].min(axis=1)
comparison_df['Best Method'] = comparison_df.apply(lambda row: 
                                                   'ARIMA' if row['Best MAE'] == row['ARIMA_MAE'] 
                                                   else 'ExpSmoothing_MAE', axis=1)

# Drop unnecessary columns
#comparison_df.drop(['ARIMA_MAE', 'ExpSmoothing_params', 'ExpSmoothing_MAE', 'LightGBM_MAE'], axis=1, inplace=True)
#comparison_df.drop(['ExpSmoothing_params'], axis=1, inplace=True)
comparison_df

# 3. DARTS TFT model

In [3]:
df.head()

,id,item_id,dept_id,state_id,sales,wday,month,year,event_name_2,snap_CA,...,event_name_1_StPatricksDay,event_name_1_SuperBowl,event_name_1_Thanksgiving,event_name_1_ValentinesDay,event_name_1_VeteransDay,event_name_1_missing,wday_sin,wday_cos,month_sin,month_cos
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,FOODS_2_197_CA_1_validation,FOODS_2_197,FOODS_2,CA,38,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025
2011-01-29,FOODS_3_080_CA_1_validation,FOODS_3_080,FOODS_3,CA,33,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025
2011-01-29,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,CA,107,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025
2011-01-29,FOODS_3_120_CA_1_validation,FOODS_3_120,FOODS_3,CA,0,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025
2011-01-29,FOODS_3_252_CA_1_validation,FOODS_3_252,FOODS_3,CA,19,1,1,0.0,missing,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.781832,0.62349,0.5,0.866025


In [4]:
from darts import TimeSeries
from darts.models.forecasting.tft_model import TFTModel
from darts.metrics import mse
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape, mae
from torchmetrics.regression import MeanAbsoluteError

/home/sljaoudli/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
product1 = df[df['id'] == 'FOODS_3_180_CA_1_validation']
product1

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
print(df.index.max(), df.index.min())

In [ ]:
# # Calculate the date 28 days before the maximum date
# last_28_start = df.index.max() - pd.Timedelta(days=28)

# # Print the last 28 days range
# print(last_28_start, "-", df.index.max())

In [ ]:
# # Calculate the date 28 days before the maximum date
# second_last_28 = last_28_start - pd.Timedelta(days=28)

# # Print the last 28 days range
# print(second_last_28, "-", last_28_start)

In [ ]:
originalindex = df.index
df.info()

In [ ]:
df['DayOfWeek'] = df.index.to_series().dt.dayofweek

In [ ]:
# from darts.dataprocessing.transformers.scaler import Scaler
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# target_scaler = MinMaxScaler()
# target_scaler.fit(df[['sales']])
# df = pd.DataFrame(scaler.fit_transform(df), columns=scaler.get_feature_names_out())
# df.index = originalindex

In [ ]:
# target = df[['sales']]
# past_cov = df.drop(columns=['sales'])
# future_cov = df.drop(columns=['sales'])

In [ ]:
target = product1[['sales']]
past_cov = product1.drop(columns=['sales', 'id','item_id','dept_id','state_id'])
future_cov = product1.drop(columns=['sales','id','item_id','dept_id','state_id'])

In [ ]:
# !pip install -q darts

In [ ]:
target.info()

In [ ]:
past_cov.info()

In [ ]:
future_cov.info()

In [ ]:
# y_train = target.loc['2011-01-29':'2016-01-29']
# past_cov_train = past_cov.loc['2011-01-29':'2016-01-29']
# future_cov_train = future_cov.loc['2011-01-29':'2016-03-26']

# y_val = target.loc['2016-01-30':'2016-03-26']
# past_cov_val = past_cov.loc['2016-01-30':'2016-03-26']
# future_cov_val = future_cov.loc['2016-01-30':'2016-04-24']

# y_test = target.loc['2016-03-27':'2016-04-24']

# y_train_backtest = target.loc['2011-01-29':'2016-03-26']
# past_cov_train_backtest = past_cov.loc['2011-01-29':'2016-03-26']
# future_cov_train_backtest = future_cov.loc['2011-01-29':'2016-04-24']

In [ ]:
y_train = target.loc[:'2016-01-01']
past_cov_train = past_cov.loc[:'2016-01-01']
future_cov_train = future_cov.loc[:'2016-01-29']

y_val = target.loc['2016-01-02':'2016-04-24']
past_cov_val = past_cov.loc['2016-01-02':'2016-04-24']
future_cov_val = future_cov.loc['2016-01-02':'2016-05-22']

y_test = target.loc['2016-04-25':'2016-05-22']

y_train_backtest = target.loc['2011-01-29':'2016-05-22']
past_cov_train_backtest = past_cov.loc['2011-01-29':'2016-05-22']
future_cov_train_backtest = past_cov.loc['2011-01-29':'2016-06-19']

In [ ]:
# from IPython.display import Image

# image_path = '../raw_data/image.png'

# # Display the image
# Image(filename=image_path)


In [ ]:
# product_data = df[df['id'] == id]

# y_train = target.loc['2011-01-29':-56]
# past_cov_train = past_cov.loc['2011-01-29':'2016-02-27']
# future_cov_train = future_cov.loc['2011-01-29':'2016-03-26']

# y_val = target.loc['2016-02-28':'2016-03-26']
# past_cov_val = past_cov.loc['2016-02-28':'2016-03-26']
# future_cov_val = future_cov.loc['2016-03-27':'2016-04-24']

# y_test = target.loc['2016-03-27':'2016-04-24']

# y_train_backtest = target.loc['2011-01-29':'2016-03-26']
# past_cov_train_backtest = past_cov.loc['2011-01-29':'2016-03-26']
# future_cov_train_backtest = future_cov.loc['2011-01-29':'2016-04-24']

In [ ]:
print(y_train.max())
print(y_train.idxmax())

In [ ]:
# # Check for duplicate timestamps in y_train DataFrame
# duplicate_indices = y_train.index.duplicated()
# print("Duplicate timestamps in y_train:", duplicate_indices.any())

In [ ]:
# # Aggregate duplicate timestamps by taking the mean
# y_train_agg = y_train.groupby(y_train.index).mean()

# # Create TimeSeries objects from the aggregated DataFrame
# y_train_series = TimeSeries.from_dataframe(y_train_agg, freq='D')


In [ ]:
# Example code assuming daily frequency ('D')
y_train_series = TimeSeries.from_dataframe(y_train, freq='D')
past_cov_train_series = TimeSeries.from_dataframe(past_cov_train, freq='D')
future_cov_train_series = TimeSeries.from_dataframe(future_cov_train, freq='D')

y_val_series = TimeSeries.from_dataframe(y_val, freq='D')
past_cov_val_series = TimeSeries.from_dataframe(past_cov_val, freq='D')
future_cov_val_series = TimeSeries.from_dataframe(future_cov_val, freq='D')

y_test_series = TimeSeries.from_dataframe(y_test, freq='D')

y_train_backtest_series = TimeSeries.from_dataframe(y_train_backtest, freq='D')
past_cov_train_backtest_series = TimeSeries.from_dataframe(past_cov_train_backtest, freq='D')
future_cov_train_backtest_series = TimeSeries.from_dataframe(future_cov_train_backtest, freq='D')


In [ ]:
y_val_series.duration, y_val_series.start_time(), y_val_series.end_time()

In [ ]:
past_cov_val_series.duration, past_cov_val_series.start_time(), past_cov_val_series.end_time()

In [ ]:
future_cov_val_series.duration, future_cov_val_series.start_time(), future_cov_val_series.end_time()

In [ ]:
y_train_series.duration, y_train_series.start_time(), y_train_series.end_time()

In [ ]:
past_cov_train_series.duration, past_cov_train_series.start_time(), past_cov_train_series.end_time()

In [ ]:
future_cov_train_series.duration, future_cov_train_series.start_time(), future_cov_train_series.end_time()

In [ ]:
# TFTModel:
input_chunk_length = 28*2
output_chunk_length = 28

In [ ]:
# series = TimeSeries.from_dataframe(df, 'date', 'sales')

# # Split the data into training, validation, and test sets
# train_val_test_split = int(len(series) * 0.7)
# train, val_test = series[:train_val_test_split], series[train_val_test_split:]
# val, test = val_test[:int(len(val_test) * 0.5)], val_test[int(len(val_test) * 0.5):]



In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# stop training when validation loss does not decrease more than 0.05 (`min_delta`) over
# a period of 5 epochs (`patience`)
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=30,
    min_delta=0.001,
    mode='min',
)

# use GPU
# pl_trainer_kwargs={"callbacks": [my_stopper],
#                    "accelerator": "gpu",
#                    "devices": [0]}

# use CPU
pl_trainer_kwargs={"callbacks": [my_stopper],
                   "accelerator": "cpu"}

In [ ]:
# Without tuning
# tft = TFTModel(input_chunk_length =input_chunk_length ,
#                output_chunk_length = output_chunk_length,
#                pl_trainer_kwargs = pl_trainer_kwargs,
#                torch_metrics=MeanAbsoluteError(),
#                n_epochs=50
#                )

# Advanced tuning
tft = TFTModel(input_chunk_length =input_chunk_length ,
               output_chunk_length = output_chunk_length,
               pl_trainer_kwargs = pl_trainer_kwargs,
               lstm_layers=2,
               num_attention_heads=4,
               dropout=0.2,
               batch_size=16,
               hidden_size=16,
               torch_metrics=MeanAbsoluteError(),
               n_epochs=1000,
               # add_encoders=add_encoders
               )

tft.fit(series=y_train_series,
        past_covariates = past_cov_train_series,
        future_covariates = future_cov_train_series,
        val_series=y_val_series,
        val_past_covariates=past_cov_val_series,
        val_future_covariates=future_cov_val_series)

In [ ]:
y_train_series.duration, y_train_series.start_time(), y_train_series.end_time()

In [ ]:
# y_val.to_numpy().reshape(168, 7, 1)

In [ ]:
preds = tft.predict(n=output_chunk_length,
                   series=y_val_series,
                   past_covariates = past_cov_val_series,
                   future_covariates = future_cov_val_series)
# score = mse(y_val_series, preds)
# preds

In [ ]:
preds.plot(label='prediction_sales')
y_test_series[:output_chunk_length].plot()

In [ ]:
mae(preds, y_test_series)

In [ ]:
future_cov_train_backtest_series.duration, future_cov_train_backtest_series.start_time(), future_cov_train_backtest_series.end_time()

In [ ]:
historical_fcast_tft = tft.historical_forecasts(
        series=y_train_backtest_series,
        past_covariates=past_cov_train_backtest_series,
        future_covariates=future_cov_train_backtest_series,
        start=0.3,
        forecast_horizon=7,
        verbose=False,
        retrain=False
)

In [ ]:
y_train_backtest_series.plot(label="data")
historical_fcast_tft.plot(low_quantile=0.01, high_quantile=0.99,label="backtest ahead forecast (TFTModel)")
print("SMAPE = {:.2f}%".format(smape(historical_fcast_tft, y_train_backtest_series)))
print("MAE = {:.2f}".format(mae(historical_fcast_tft, y_train_backtest_series)))

In [ ]:
pred_values = target_scaler.inverse_transform(preds.pd_dataframe())
real_values = target_scaler.inverse_transform(y_test.values)

In [ ]:
def smape_function(actual, predicted):
    """
    Calculate Symmetric Mean Absolute Percentage Error (SMAPE) between two arrays.

    Parameters:
    - actual: array containing actual values
    - predicted: array containing predicted values

    Returns:
    - SMAPE value
    """
    denominator = (np.abs(actual) + np.abs(predicted)) / 2.0
    diff = np.abs(actual - predicted) / denominator
    diff[denominator == 0] = 0.0  # Handle division by zero
    smape_value = np.mean(diff) * 100.0

    return smape_value

In [ ]:
smape_function(pred_values, real_values[:output_chunk_length])

In [ ]:
from darts.explainability.tft_explainer import TFTExplainer

explainer = TFTExplainer(tft)
results = explainer.explain()
# plot the results
# explainer.plot_attention(results, plot_type="heatmap")
explainer.plot_variable_selection(results)

In [5]:
from darts import TimeSeries
from darts.models.forecasting.tft_model import TFTModel
from darts.metrics import mse
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape, mae
from torchmetrics.regression import MeanAbsoluteError

In [6]:
def prepare_data(product_data):
    target = product_data[['sales']]
    past_cov = product_data.drop(columns=['sales', 'id','item_id','dept_id','state_id','event_name_2'])
    future_cov = product_data.drop(columns=['sales','id','item_id','dept_id','state_id','event_name_2'])

    y_train = target.loc[:'2016-01-01']
    past_cov_train = past_cov.loc[:'2016-01-01']
    future_cov_train = future_cov.loc[:'2016-01-29']

    y_val = target.loc['2016-01-02':'2016-04-24']
    past_cov_val = past_cov.loc['2016-01-02':'2016-04-24']
    future_cov_val = future_cov.loc['2016-01-02':'2016-05-22']

    return (y_train, past_cov_train, future_cov_train,
            y_val, past_cov_val, future_cov_val)

In [7]:
def train_tft_model(y_train_series, past_cov_train_series, future_cov_train_series,
                    y_val_series, past_cov_val_series, future_cov_val_series):
    input_chunk_length = 28*2
    output_chunk_length = 28

    from pytorch_lightning.callbacks.early_stopping import EarlyStopping

    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=30,
        min_delta=0.001,
        mode='min',
    )

    pl_trainer_kwargs={"callbacks": [my_stopper],
                       "accelerator": "cpu"}

    tft = TFTModel(input_chunk_length=input_chunk_length,
                   output_chunk_length=output_chunk_length,
                   pl_trainer_kwargs=pl_trainer_kwargs,
                   lstm_layers=2,
                   num_attention_heads=4,
                   dropout=0.2,
                   batch_size=16,
                   hidden_size=16,
                   torch_metrics=MeanAbsoluteError(),
                   n_epochs=500,
                   )

    tft.fit(series=y_train_series,
            past_covariates=past_cov_train_series,
            future_covariates=future_cov_train_series,
            val_series=y_val_series,
            val_past_covariates=past_cov_val_series,
            val_future_covariates=future_cov_val_series)

    return tft

In [ ]:
results_dict = {}

# Iterate over each unique product series identified by id 
for product_id in df['id'].unique()[:10]:
    print(f"Training model for product: {product_id}")
    
    # Extract data for the current product
    product_data = df[df['id'] == product_id]
    
    # Prepare data
    (y_train, past_cov_train, future_cov_train,
     y_val, past_cov_val, future_cov_val) = prepare_data(product_data)

    # Example code assuming daily frequency ('D')
    y_train_series = TimeSeries.from_dataframe(y_train, fill_missing_dates=True, freq='D')
    past_cov_train_series = TimeSeries.from_dataframe(past_cov_train, fill_missing_dates=True, freq='D')
    future_cov_train_series = TimeSeries.from_dataframe(future_cov_train, fill_missing_dates=True, freq='D')

    y_val_series = TimeSeries.from_dataframe(y_val, fill_missing_dates=True, freq='D')
    past_cov_val_series = TimeSeries.from_dataframe(past_cov_val, fill_missing_dates=True, freq='D')
    future_cov_val_series = TimeSeries.from_dataframe(future_cov_val, fill_missing_dates=True, freq='D')

    # Train model
    trained_model = train_tft_model(y_train_series, past_cov_train_series, future_cov_train_series,
                                    y_val_series, past_cov_val_series, future_cov_val_series)

    # Store the trained model in the dictionary
    results_dict[product_id] = {'model': trained_model, 'forecast': trained_model.predict(n=28)}


Training model for product: FOODS_2_197_CA_1_validation


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sljaoudli/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 73.5 K
5  | decoder_vsn                       | _VariableSelect

Sanity Checking DataLoader 0:   0%|                                                               | 0/2 [00:00<?, ?it/s]

/home/sljaoudli/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/pytorch_lightning/core/module.py:507: You called `self.log('val_MeanAbsoluteError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Epoch 0:   0%|                                                                                  | 0/108 [00:00<?, ?it/s]

...

Sanity Checking DataLoader 0:   0%|                                                               | 0/2 [00:00<?, ?it/s]

/home/sljaoudli/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/pytorch_lightning/core/module.py:507: You called `self.log('val_MeanAbsoluteError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Epoch 0:   0%|                                                                                  | 0/108 [00:00<?, ?it/s]

/home/sljaoudli/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/pytorch_lightning/core/module.py:507: You called `self.log('train_MeanAbsoluteError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Epoch 0:   6%|███                                                     | 6/108 [00:03<00:51,  1.96it/s, train_loss=224.0]